This notebook helps to prepare a training data for the CTD<sup>2</sup> Pancancer Drug Activity DREAM Challenge.

Created by: Bence Szalai, Semmelweis University, Budapest, Hungary

We will use the gene expression profiles from [LINCS-L1000 project](https://www.sciencedirect.com/science/article/pii/S0092867417313090). The compound metadata is coming from [The Drug Repurposing Hub](https://www.nature.com/articles/nm.4306). The analysis used here is part of [our previous manuscript](https://academic.oup.com/nar/article/47/19/10010/5573547?guestAccessKey=151831e1-0f78-4c70-854c-a471a28a5eac). We will also use the [cmapPy library](https://academic.oup.com/bioinformatics/article/35/8/1427/5094509). If you use this analyis in your work / following manuscript (after the challange publication embargo) please cite all of them appropriately. 

You have to download the data from Gene Expression Omnibus. We will use [GSE92742](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE92742) and [GSE70138](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE70138) datasets. You have to download from both datasets the following files:

* Level5 gene expression profiles (GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx.gz and GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328_2017-03-06.gctx.gz respectively)
* metadata from sig_info files (GSE92742_Broad_LINCS_sig_info.txt.gz and GSE70138_Broad_LINCS_sig_info_2017-03-06.txt.gz respectively)
* gene info file (GSE92742_Broad_LINCS_gene_info.txt.gz, no need to download it from GSE70138, they are the same)

Please download (and `gunzip`) these files, and put their content into the **data/GSE92742** and **data/GSE70138** folders of this project.

You will also need compound metadata from [The Drug Repurposing Hub](https://clue.io/repurposing). You have to download the following files:
* Drug information (updated 9/7/2018)
* Sample information (updated 9/7/2018)

Please download these files and put them to **data/repurposing**.

In [1]:
#let's check files are in the correct directory
import os
files_needed_GSE92742=['GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
               'GSE92742_Broad_LINCS_sig_info.txt','GSE92742_Broad_LINCS_gene_info.txt']
files_needed_GSE70138=['GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328.gctx',
                      'GSE70138_Broad_LINCS_sig_info.txt']
files_needed_rep=['repurposing_drugs_20180907.txt','repurposing_samples_20180907.txt']
files_GSE92742=os.listdir('../data/GSE92742/')
files_GSE70138=os.listdir('../data/GSE70138/')
files_rep=os.listdir('../data/repurposing/')
for f in files_needed_GSE92742:
    assert (f in files_GSE92742)
for f in files_needed_GSE70138:
    assert (f in files_GSE70138)
for f in files_needed_rep:
    assert (f in files_rep)

You will need to install the [cmapPy library](https://clue.io/cmapPy/build.html#install).

In [2]:
# general data manipulations
import pandas as pd 
import numpy as np

#cmapPy
from cmapPy.pandasGEXpress.parse import parse

/Users/benceszalai/Library/Enthought/Canopy/edm/envs/Python3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


First we will get familiar with LINCS-L1000 data, and get shRNA related perturabtions.

In [3]:
#this files describe the perturbation metadata.
#sig_id (index) is unique to a given measurement
sig_info_gse92742=pd.read_csv('../data/GSE92742/GSE92742_Broad_LINCS_sig_info.txt',
                             sep='\t',header=0,index_col=0,low_memory=False)
sig_info_gse92742.head()

,pert_id,pert_iname,pert_type,cell_id,pert_dose,pert_dose_unit,pert_idose,pert_time,pert_time_unit,pert_itime,distil_id
sig_id,,,,,,,,,,,
AML001_CD34_24H:A05,DMSO,DMSO,ctl_vehicle,CD34,0.1,%,0.1 %,24,h,24 h,AML001_CD34_24H_X1_F1B10:A05
AML001_CD34_24H:A06,DMSO,DMSO,ctl_vehicle,CD34,0.1,%,0.1 %,24,h,24 h,AML001_CD34_24H_X3_F1B10:A06
AML001_CD34_24H:B05,DMSO,DMSO,ctl_vehicle,CD34,0.1,%,0.1 %,24,h,24 h,AML001_CD34_24H_X1_F1B10:B05|AML001_CD34_24H_X...
AML001_CD34_24H:B06,DMSO,DMSO,ctl_vehicle,CD34,0.1,%,0.1 %,24,h,24 h,AML001_CD34_24H_X3_F1B10:B06
AML001_CD34_24H:BRD-A03772856:0.37037,BRD-A03772856,BRD-A03772856,trt_cp,CD34,0.37037,µM,500 nM,24,h,24 h,AML001_CD34_24H_X1_F1B10:J04|AML001_CD34_24H_X...


In [4]:
#we select measurements where shRNA was used to knock-down a given gene
#we will focuse singatures where pert_type column is a trt_sh.cgs, thw consensus signature of gene KD
fil=sig_info_gse92742['pert_type']=='trt_sh.cgs'
sig_info_shRNA=sig_info_gse92742[fil].copy()
sig_info_shRNA.head()
#here pert_iname is the knock-downed gene, cell_id is the cell line

,pert_id,pert_iname,pert_type,cell_id,pert_dose,pert_dose_unit,pert_idose,pert_time,pert_time_unit,pert_itime,distil_id
sig_id,,,,,,,,,,,
CGS001_A375_96H:A2M:1,CGS001-2,A2M,trt_sh.cgs,A375,1.0,µL,1 µL,96,h,96 h,KDC004_A375_96H:TRCN0000006653:-666|KDC004_A37...
CGS001_A375_96H:AARS:1,CGS001-16,AARS,trt_sh.cgs,A375,1.0,µL,1 µL,96,h,96 h,KDB006_A375_96H:TRCN0000045688:-666|KDB006_A37...
CGS001_A375_96H:AATF:1,CGS001-26574,AATF,trt_sh.cgs,A375,1.0,µL,1 µL,96,h,96 h,KDA004_A375_96H:TRCN0000017389:-666|KDA004_A37...
CGS001_A375_96H:ABAT:1,CGS001-18,ABAT,trt_sh.cgs,A375,1.0,µL,1 µL,96,h,96 h,KDA002_A375_96H:TRCN0000034925:-666|KDB006_A37...
CGS001_A375_96H:ABCA1:1,CGS001-19,ABCA1,trt_sh.cgs,A375,1.0,µL,1 µL,96,h,96 h,KDC004_A375_96H:TRCN0000029089:-666|KDC004_A37...


In [5]:
#we will use only the measured, landmark genes (for simplicity, and we also thrust measured genes more then infered)
gene_ids=pd.read_csv('../data/GSE92742/GSE92742_Broad_LINCS_gene_info.txt',
                    sep='\t',header=0,index_col=0)
gene_ids.head()
#pr_gene_id is the gene id used by LINCS,pr_gene_symbol is the 'human readable' gene symbol
#pr_is_lm is 1 for measured genes

,pr_gene_symbol,pr_gene_title,pr_is_lm,pr_is_bing
pr_gene_id,,,,
780,DDR1,discoidin domain receptor tyrosine kinase 1,1,1
7849,PAX8,paired box 8,1,1
2978,GUCA1A,guanylate cyclase activator 1A,0,0
2049,EPHB3,EPH receptor B3,0,1
2101,ESRRA,estrogen related receptor alpha,0,1


In [6]:
fil=gene_ids['pr_is_lm']==1
gene_ids=gene_ids[fil]
gene_ids=gene_ids['pr_gene_symbol']
gene_ids.head()

pr_gene_id
780      DDR1
7849     PAX8
6193     RPS5
23      ABCF1
9552    SPAG7
Name: pr_gene_symbol, dtype: object

In [7]:
#an example to select perturbations where A2M gene was knocked-down
fil=sig_info_shRNA['pert_iname']=='A2M'
samples=sig_info_shRNA.index[fil]
#cid is the sample ides, while rid is the gene ids
try:
    gene_ids.index=gene_ids.index.astype(str) #in some version gene ids are string
    expression=parse('../data/GSE92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
                 cid=samples,rid=gene_ids.index).data_df.T[gene_ids.index]
except:
    gene_ids.index=gene_ids.index.astype(int) # in others integer
    expression=parse('../data/GSE92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
                 cid=samples,rid=gene_ids.index).data_df.T[gene_ids.index]
expression.head()
#so here the rows are the samples where the given genes were knocked down
#columsn are the measured gene expression changes

rid,780,7849,6193,23,9552,387,10921,10285,533,6194,...,54681,11000,6915,6253,7264,5467,2767,23038,57048,79716
cid,,,,,,,,,,,,,,,,,,,,,
CGS001_A375_96H:A2M:1,-0.450660,0.613190,-0.052799,-0.690646,-0.884775,-0.268257,0.989756,-0.270924,0.562012,-0.064880,...,0.769396,-0.574408,0.037353,-0.094501,0.246233,0.055728,-0.032736,0.554488,0.042339,-0.875408
CGS001_A549_96H:A2M:1,-1.007425,-0.465669,-0.152609,-0.854945,-0.173800,0.701383,0.440690,0.388689,0.346691,0.001048,...,0.821364,-1.732306,-0.990096,-2.820324,0.476273,-0.699089,0.398378,0.025279,0.423876,-1.017771
CGS001_HA1E_96H:A2M:1.5,-0.487499,0.373099,0.906173,-0.472660,-0.973289,1.075552,-0.182322,-0.518891,0.030982,-0.204590,...,-0.289508,0.623188,-1.103047,-0.815189,0.651688,-0.208761,0.345416,0.606831,1.363954,-1.080051
CGS001_HEPG2_96H:A2M:1.5,-0.577616,-0.594645,0.088793,-0.679613,-1.105863,0.353294,0.151503,-0.299633,-0.678864,-0.074211,...,-0.222315,-0.324444,-0.194537,-1.031143,0.009928,-0.348788,0.813288,1.007242,1.392832,0.153785
CGS001_HT29_96H:A2M:1,-0.272184,-0.172414,0.200608,-0.235290,-0.262455,0.432267,-0.113109,-0.146903,-0.888957,0.007347,...,-0.056524,-0.510606,-0.666978,-0.377540,0.178874,0.180162,1.115406,0.154438,0.346749,-0.103932


In this analysis we won't work with the individual signatures, but will caculate a consensus (average) signature for a given gene knockout. However further filtering / using of individual signatures are highly encouraged. To calculate consensus signature, we will use the MODZ method described in the original LINCS manuscript.

In [8]:
from scipy.stats import spearmanr as scor
def calc_MODZ(data):
    """calculates MODZ based on the original CMAP/L1000 study
    use only lm genes for MODZ calculation! Uses LM_GENES global
    variable."""
    if len(data)==1:
        return data.iloc[0]
    if len(data)==2:
        return np.mean(data,0)
    else:
        CM=scor(data.T)[0]
        fil=CM<0
        CM[fil]=0.01
        weights=np.sum(CM,1)-1
        weights=weights/np.sum(weights)
        weights=weights.reshape((-1,1))
        return pd.Series(np.dot(data.T,weights).reshape((-1,1)[0]),index=data.columns)

In [9]:
#so this is the consensus signature of the A2M gene knockdown
calc_MODZ(expression).head()

rid
780    -0.370345
7849   -0.021135
6193    0.446688
23     -0.631813
9552   -0.688607
dtype: float64

In [18]:
#we will store perturabtion expression profiles in a DataFrame
#rows are the perturbed genes, columns are the measured landmark genes from LINCS
genes_perturbed=sig_info_shRNA['pert_iname'].unique()
consensus_signatures_shRNA=pd.DataFrame(index=genes_perturbed,columns=gene_ids.index)
consensus_signatures_shRNA.head()

pr_gene_id,780,7849,6193,23,9552,387,10921,10285,533,6194,...,54681,11000,6915,6253,7264,5467,2767,23038,57048,79716
A2M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AARS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AATF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABCA1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
for i in range(len(genes_perturbed)):
    if (i%100)==0:
        print('Done for %i genes' %i)
    gene=genes_perturbed[i]
    fil=sig_info_shRNA['pert_iname']==gene
    samples=sig_info_shRNA.index[fil]
    expression=parse('../data/GSE92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
                 cid=samples,rid=gene_ids.index).data_df.T[gene_ids.index]
    consensus_signatures_shRNA.loc[gene]=calc_MODZ(expression)

Done for 0 genes
Done for 100 genes
Done for 200 genes
Done for 300 genes
Done for 400 genes
Done for 500 genes
Done for 600 genes
Done for 700 genes
Done for 800 genes
Done for 900 genes
Done for 1000 genes
Done for 1100 genes
Done for 1200 genes
Done for 1300 genes
Done for 1400 genes
Done for 1500 genes
Done for 1600 genes
Done for 1700 genes
Done for 1800 genes
Done for 1900 genes
Done for 2000 genes
Done for 2100 genes
Done for 2200 genes
Done for 2300 genes
Done for 2400 genes
Done for 2500 genes
Done for 2600 genes
Done for 2700 genes
Done for 2800 genes
Done for 2900 genes
Done for 3000 genes
Done for 3100 genes
Done for 3200 genes
Done for 3300 genes
Done for 3400 genes
Done for 3500 genes
Done for 3600 genes
Done for 3700 genes
Done for 3800 genes
Done for 3900 genes
Done for 4000 genes
Done for 4100 genes
Done for 4200 genes
Done for 4300 genes


In [25]:
#rename LINCS gene ids for gene symbolc
consensus_signatures_shRNA.columns=gene_ids[consensus_signatures_shRNA.columns].values

In [27]:
consensus_signatures_shRNA.to_csv('../results/consensus_signature_shRNA.csv',sep=',')

So you can use the **consensus_signature_shRNA.csv** file for your models. The same file is also available in Synapse. While this data only contains consensus signatures, further experiments with individual signatures can further increase model perferomance, and are highly encouraged.

Now we will do the same consensus signature calculations for drugs. We will use both GSE92742 and GSE70138.

In [85]:
sig_info_gse92742=pd.read_csv('../data/GSE92742/GSE92742_Broad_LINCS_sig_info.txt',
                              sep='\t',header=0,index_col=0,low_memory=False)
sig_info_gse70138=pd.read_csv('../data/GSE70138/GSE70138_Broad_LINCS_sig_info.txt',
                              sep='\t',header=0,index_col=0,low_memory=False)

In [90]:
#keep only compoud treatments
fil=sig_info_gse92742['pert_type']=='trt_cp'
sig_info_gse92742=sig_info_gse92742[fil]
fil=sig_info_gse70138['pert_type']=='trt_cp'
sig_info_gse70138=sig_info_gse70138[fil]

In [92]:
#get all compounds
cpds=list(set(sig_info_gse70138['pert_id']) | set(sig_info_gse92742['pert_id']))

To get the targets of these drugs - when target is known - we need compound metadata from Drug Repurposing Hub.

In [117]:
# read drug repurposing data
repurposing=pd.read_csv('../data/repurposing/repurposing_drugs_20180907.txt',sep='\t',encoding='latin',
                    header=0,index_col=None,skiprows=9)
repurposing_samples=pd.read_csv('../data/repurposing/repurposing_samples_20180907.txt',sep='\t',encoding='latin',
                    header=0,index_col=None,skiprows=9)

In [118]:
#correct broad_id
repurposing_samples['broad_id']=repurposing_samples['broad_id'].apply(lambda x: '-'.join(x.split('-')[:2]))
#select usefull columns
repurposing_samples=repurposing_samples[['broad_id','pert_iname']]
repurposing=repurposing[['pert_iname','moa','target']]

In [119]:
#merge repurposing data
drug_metadata=pd.merge(repurposing,repurposing_samples)
#remove duplicated
drug_metadata=drug_metadata.drop_duplicates()
#keep only rows with known target / MoA
fil=~(drug_metadata['moa'].isnull() & drug_metadata['target'].isnull())
drug_metadata=drug_metadata[fil]

In [120]:
#matching compounds between repurposing and LINCS dataset
cpds=list(set(cpds) & set(drug_metadata['broad_id']))
fil=np.in1d(drug_metadata['broad_id'],cpds)
drug_metadata=drug_metadata[fil]
drug_metadata.index=range(len(drug_metadata))

In [121]:
#select all targets / MoAs
all_moas=[]
all_targets=[]
for i in drug_metadata.index:
    if not pd.isnull(drug_metadata.loc[i,'target']):
        all_targets+=drug_metadata.loc[i,'target'].split('|')
    if  not pd.isnull(drug_metadata.loc[i,'moa']):
        all_moas+=drug_metadata.loc[i,'moa'].split('|')
all_moas=list(set(all_moas))
all_targets=list(set(all_targets))

In [122]:
meta_matrix=pd.DataFrame(0,index=list(set(drug_metadata['broad_id'])),columns=all_moas+all_targets)

In [124]:
#for compunds where inhibitor,blocker,antagonist etc. are in the MoA columns
#we assume they are inhibitory compounds, so we will mark them with -1 in the meta_matrix
# for other compounds, we assume they are activators, we will mark them with +1
#this is probably not a perfect way to access inhibitory/acovatory state, but good for a first try
inhibitory_words=set(['inhibitor','blocker','antagonist','inihibitor']) #inihibitor is just a typo
for i in drug_metadata.index:
    if list(drug_metadata.index).index(i) % 100==0:
        print('Done for %i drugs' %list(drug_metadata.index).index(i))
    brd=drug_metadata.loc[i,'broad_id']
    if not pd.isnull(drug_metadata.loc[i,'moa']):
        moas=drug_metadata.loc[i,'moa'].split('|')
    else:
        moas=[]
    if not pd.isnull(drug_metadata.loc[i,'target']):
        s=1
        targets=drug_metadata.loc[i,'target'].split('|')
        if len(set((' '.join(moas)).split())&inhibitory_words)>0:
            s=-1
    else:
        targets=[]
    meta_matrix.loc[brd,moas]=1
    meta_matrix.loc[brd,targets]=s

Done for 0 drugs
Done for 100 drugs
Done for 200 drugs
Done for 300 drugs
Done for 400 drugs
Done for 500 drugs
Done for 600 drugs
Done for 700 drugs
Done for 800 drugs
Done for 900 drugs
Done for 1000 drugs
Done for 1100 drugs
Done for 1200 drugs
Done for 1300 drugs
Done for 1400 drugs
Done for 1500 drugs
Done for 1600 drugs
Done for 1700 drugs
Done for 1800 drugs
Done for 1900 drugs
Done for 2000 drugs
Done for 2100 drugs
Done for 2200 drugs
Done for 2300 drugs
Done for 2400 drugs
Done for 2500 drugs
Done for 2600 drugs
Done for 2700 drugs
Done for 2800 drugs


In [125]:
meta_matrix.to_csv('../results/drugs_meta.csv',sep=',')

Now we will get the consensus signature for each drug, and store them in a dataframe.

In [142]:
consensus_signatures_drugs=pd.DataFrame(index=meta_matrix.index,columns=gene_ids.index)

In [143]:
for i in range(len(consensus_signatures_drugs.index)):
    drug=consensus_signatures_drugs.index[i]
    if i%100==0:
        print('Done for %i drugs' % i)
    fil=sig_info_gse70138['pert_id']==drug
    if fil.sum()>0:
        samples=sig_info_gse70138.index[fil]
        expression_gse70138=parse('../data/GSE70138/GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328.gctx',
                                  cid=samples,rid=gene_ids.index).data_df.T[gene_ids.index]
    else:
        expression_gse70138=pd.DataFrame(columns=gene_ids.index)
    fil=sig_info_gse92742['pert_id']==drug
    if fil.sum()>0:
        samples=sig_info_gse92742.index[fil]
        expression_gse92742=parse('../data/GSE92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx',
                                  cid=samples,rid=gene_ids.index).data_df.T[gene_ids.index]
    else:
        expression_gse92742=pd.DataFrame(columns=gene_ids.index)
    expression=pd.concat([expression_gse70138,expression_gse92742])
    consensus_signatures_drugs.loc[drug]=calc_MODZ(expression)
        
        

Done for 0 drugs
Done for 100 drugs
Done for 200 drugs
Done for 300 drugs
Done for 400 drugs
Done for 500 drugs
Done for 600 drugs
Done for 700 drugs
Done for 800 drugs
Done for 900 drugs
Done for 1000 drugs
Done for 1100 drugs
Done for 1200 drugs
Done for 1300 drugs
Done for 1400 drugs
Done for 1500 drugs
Done for 1600 drugs
Done for 1700 drugs
Done for 1800 drugs
Done for 1900 drugs
Done for 2000 drugs
Done for 2100 drugs
Done for 2200 drugs
Done for 2300 drugs
Done for 2400 drugs
Done for 2500 drugs
Done for 2600 drugs
Done for 2700 drugs
Done for 2800 drugs


In [150]:
#rename L1000 gene ids for gene names
consensus_signatures_drugs.columns=gene_ids[consensus_signatures_drugs.columns].values

In [151]:
consensus_signatures_drugs.to_csv('../results/consensus_signature_drugs.csv',sep=',')